In [1]:
import os
import numpy as np
from scipy import interpolate
import scipy.io
import Rbeast as rb
from SLM_tools import *
import pickle
import re

In [2]:
def create_file_dicts(folder_path):
    """
    Iterates over all .mat files in the given folder path, extracts type, mu, and run_num,
    and creates dictionaries for distance and energy files with (mu, run_num) as keys.
    
    :param folder_path: Path to the folder
    :return: Tuple containing two dictionaries (distance_dict, energy_dict)
    """
    distance_dict = {}
    energy_dict = {}
    pattern = re.compile(
        r"(?P<type>distance|energy)_vec_mu_(?P<mu>[-+]?\d*\.\d+|\d+)_energy_(?P<energy>[-+]?\d*\.\d+|\d+)_run_num_(?P<run_num>\d+)_total_num_target_\d+\.mat")

    try:
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path) and filename.endswith('.mat'):
                match = pattern.match(filename)
                if match:
                    file_info = match.groupdict()
                    key = (float(file_info['mu']), int(file_info['run_num']))
                    if file_info['type'] == 'distance':
                        distance_dict[key] = filename
                    elif file_info['type'] == 'energy':
                        energy_dict[key] = filename
    except Exception as e:
        print(f"An error occurred: {e}")

    return distance_dict, energy_dict

def load_and_concatenate_files_horizontally(distance_dict, energy_dict, folder_path):
    """
    Loads and concatenates files with the same keys in the distance and energy dictionaries.
    Concatenates the data horizontally, placing energy columns before distance columns.
    
    :param distance_dict: Dictionary containing distance files with keys as (mu, run_num)
    :param energy_dict: Dictionary containing energy files with keys as (mu, run_num)
    :param folder_path: Path to the folder containing the files
    :return: Dictionary with keys as (mu, run_num) and concatenated data as values
    """
    concatenated_data = {}

    for key in distance_dict.keys():
        if key in energy_dict:
            distance_file = os.path.join(folder_path, distance_dict[key])
            energy_file = os.path.join(folder_path, energy_dict[key])

            # Load the .mat files
            distance_data = scipy.io.loadmat(distance_file)['foo']
            energy_data = scipy.io.loadmat(energy_file)['foo'].T
            concatenated_data[key] = np.concatenate((energy_data, distance_data), axis=1)
            # print(f"energy_shape {energy_data.shape} , distance_shape {distance_data.shape}")
    return concatenated_data

In [3]:
folder_path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\Final Project\Project2-Omri and Idan\Results\06_30_2022_10_12_02_mu_0__0_2__2_8_Js__4_0_runs_2\2\3"
distance_dict, energy_dict = create_file_dicts(folder_path)

In [4]:
distance_dict = {k: v for k, v in distance_dict.items() if k[0] == 0.6}
energy_dict = {k: v for k, v in energy_dict.items() if k[0] == 0.6}

In [5]:
print(distance_dict)

{(0.6, 1): 'distance_vec_mu_0.6_energy_4.0_run_num_1_total_num_target_2.mat',
 (0.6, 2): 'distance_vec_mu_0.6_energy_4.0_run_num_2_total_num_target_2.mat'}

In [ ]:
concatenated_data = load_and_concatenate_files_horizontally(distance_dict, energy_dict, folder_path)
output_folder = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\testing data"
for key, data in concatenated_data.items():
    mu, run_num = key
    filename = f"energy_distance_mu_{mu}_run_num_{run_num}.mat"
    output_file = os.path.join(output_folder, filename)
    sio.savemat(output_file, {"filename":data})
    print(f"Saved {output_file}")